### Imports

In [124]:
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.image import imread, imsave
from pathlib import Path

### Path to images

In [38]:
dir_imgs     = Path('../../mnt/c/Users/dimit/Downloads/unlabeled_training_data/unlabeled_data/')
dir_new_imgs = dir_imgs.parent.joinpath('labeled_data')
dir_new_imgs.mkdir()

### Loading the first unlabeled image

In [45]:
img_path = list(dir_imgs.iterdir())[0] # [0] -> 0FRGJD.png
img = imread(img_path)

### Sliding window

In [81]:
size      = 256
step      = int(0.8*size)
img_count = 1

for i in np.arange(0, img.shape[0], step):
    for j in np.arange(0, img.shape[1], step):
        if i+size > img.shape[0]:
            continue
        if j+size > img.shape[1]:
            continue
        new_img = img[i:i+size, j:j+size, :].copy()
        imsave(dir_new_imgs.joinpath('{}.png'.format(img_count)), new_img, format='png')
        img_count += 1

### Arrays with truth labels

Need to do this manually and carefully... 

In [87]:
background_imgs  = np.arange(1, img_count)
trampoline_imgs  = np.array([504, 549, 550, 662, 668, 695, 706, 730, 733, 819, 820, 851, 1006, 1012, 1013, 1022, 1043, 1049, 1050, 1085, 1089, 1094, 1119, 1132, 1165, 1170, 1202, 1217, 1218, 1238, 1239, 1240, 1287, 1288, 1332, 1359])
pound_imgs       = np.array([542, 725, 736, 774, 885, 886, 923, 924, 933, 965, 996, 1009, 1045, 1163, 1256, 1285, 1319, 1432])
pool_imgs        = np.array([882])
solar_panel_imgs = np.array([821, 823, 824, 859, 860, 863, 896, 919, 920, 932, 939, 957, 1004, 1036, 1041, 1051, 1069, 1071, 1074, 1172, 1175, 1176, 1213, 1214, 1215, 1284, 1322])
confusing_imgs   = np.array([511, 512, 547, 582, 822, 858, 895, 958, 971, 984, 1005, 1037, 1046, 1047, 1075, 1201, 1210, 1237, 1324, 1325, 1353, 1354, 1357, 1358, 1391, 1392, 1396, 1433])

### Removing pools, trampolines, pounds, solar and confusing images from the background class

In [103]:
background_imgs = list(filter(lambda i: i not in trampoline_imgs, background_imgs))
background_imgs = list(filter(lambda i: i not in pound_imgs, background_imgs))
background_imgs = list(filter(lambda i: i not in pool_imgs, background_imgs))
background_imgs = list(filter(lambda i: i not in solar_panel_imgs, background_imgs))
background_imgs = list(filter(lambda i: i not in confusing_imgs, background_imgs))
background_imgs = np.array(background_imgs)
assert len(background_imgs) == img_count - 1 - (len(trampoline_imgs) + len(pound_imgs) + len(pool_imgs) + len(solar_panel_imgs) + len(confusing_imgs))

### Downsampling the background class

In [120]:
np.random.seed(0)
background_imgs_downsampled = np.random.choice(background_imgs, 100, replace=False)

### Moving files to organized paths

In [131]:
classes = {
    'background': background_imgs_downsampled,
    'pools':      pool_imgs,
    'pounds':     pound_imgs,
    'solar':      solar_panel_imgs,
    'trampoline': trampoline_imgs
}

for folder_name, array in classes.items():
    class_path = dir_new_imgs.joinpath(folder_name)
    class_path.mkdir()
    for img_number in array:
        img_name  = '{}.png'.format(img_number)
        from_path = dir_new_imgs.joinpath(img_name)
        to_path   = class_path.joinpath(img_name)
        shutil.move(from_path, to_path)

### Deleting samples that were not selected to the background class

In [133]:
for item in dir_new_imgs.iterdir():
    if item.is_file():
        item.unlink()

### Review images

Need to do this manually and carefully again...

In [134]:
not_sure_background = np.array([394, 507, 508, 545, 546, 945, 981, 1023, 1296])
not_sure_trampoline = np.array([549, 550, 1012, 1013, 1049])
not_sure_pound      = np.array([923, 924, 996, 1009])
not_sure_pool       = np.array([])
not_sure_solar      = np.array([1071])

### Removing images I'm not sure

In [136]:
classes = {
    'background': not_sure_background,
    'pools':      not_sure_pool,
    'pounds':     not_sure_pound,
    'solar':      not_sure_solar,
    'trampoline': not_sure_trampoline
}

for folder_name, array in classes.items():
    class_path = dir_new_imgs.joinpath(folder_name)
    for img_number in array:
        img_path = class_path.joinpath('{}.png'.format(img_number))
        img_path.unlink()